In [1]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

from stop_words_list import stop_words

In [2]:
pd.set_option('max_colwidth', 400)

In [3]:
import os
current_path = os.path.abspath(os.getcwd())
plots_directory = os.path.join(current_path, 'datasets/transcripts')

if not os.path.exists(plots_directory):
    os.makedirs(plots_directory)

In [4]:
connection = create_connection()

In [5]:
df = pd.read_csv('datasets/1_clean_data.csv')
df.head(1)

,Unnamed: 0,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-w7xC.html,2,0,1,0,263,2,1


In [6]:
def url_to_transcript(url):
    page = requests.get(url).text
    bs = BeautifulSoup(page, "lxml")
    container = bs.find(class_="main-content")
    article = container.find("article")
#     article.find("p", {"class": "playerBoard__text playerBoard__text--icon"}).decompose()
#     article.find("p", {"class": "playerBoard__text playerBoard__title"}).decompose()
#     article.find("p", {"class": "playerBoard__text"}).decompose()
    try:
        article.find("p", {"class": "playerBoard__text playerBoard__text--icon"}).decompose()
    except:
        print("NO CLASS: playerBoard__text playerBoard__text--icon")

    try:
        article.find("p", {"class": "playerBoard__text playerBoard__title"}).decompose()
    except:
        print("NO CLASS: playerBoard__text playerBoard__title")

    try:
        article.find("p", {"class": "playerBoard__text"}).decompose()
    except:
        print("NO CLASS: playerBoard__text")
    texts = article.find_all("p")
    articles = []
    for t in texts:
        article = t.text
        articles.append(article)
    return articles

In [7]:
data = {}

In [8]:
names = ["2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08"]

In [9]:
# for name in names:
#     name_urls = df[['url']].where(df['publication_month'] == name).dropna()
#     text = []
#     for key, url in name_urls.items():
#         for k, v in url.items():
#             text.append(url_to_transcript(v))
#     text_string = ''.join([str(item) for item in text])
#     data[name] = text_string

In [10]:
# for i, name in enumerate(names):
#     with open("datasets/transcripts/" + name + ".txt", "wb") as file:
#         pickle.dump(data[name], file)

In [11]:
data = {}
for i, name in enumerate(names):
    with open("datasets/transcripts/" + name + ".txt", "rb") as file:
        data[name] = pickle.load(file)

In [12]:
data.keys()

dict_keys(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08'])

In [13]:
data['2020-01']

'[\'Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych może usprawnić ten proces. Testy przebiegły pomyślnie. Co się zmieni?\', \'Zakład Ubezpieczeń Społecznych udostępnił medykom nowe narzędzie do wystawiania elektronicznych zwolnień, które pod koniec 2018 roku zastąpiły papierowe druki. Wszystkie ważne informacje i pola do wypełnienia znajdują się teraz na jednej stronie.\', \'– Formularz do wystawiania elektronicznych zwolnień lekarskich jest teraz dla lekarzy znacznie prostszy, bardziej przejrzysty i funkcjonalny. Dodatkowo lekarz ma wygodny podgląd do listy wcześniej wystawionych zwolnień, jakie pacjent otrzymał w ostatnich miesiącach – tłumaczy Małgorzata Korba, regionalny rzecznik ZUS na Lubelszczyźnie.\', \'Testy przyniosły pozytywne opinie. Lekarze z regionu znajdą nowy kreator na swojej Platformie Usług Elektronicznych.\', \'Na PUE ZUS pojawiły się również inne nowe funkcje dla lekarzy

In [14]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [15]:
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [16]:
data_combined['2020-01']

['[\'Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych może usprawnić ten proces. Testy przebiegły pomyślnie. Co się zmieni?\', \'Zakład Ubezpieczeń Społecznych udostępnił medykom nowe narzędzie do wystawiania elektronicznych zwolnień, które pod koniec 2018 roku zastąpiły papierowe druki. Wszystkie ważne informacje i pola do wypełnienia znajdują się teraz na jednej stronie.\', \'– Formularz do wystawiania elektronicznych zwolnień lekarskich jest teraz dla lekarzy znacznie prostszy, bardziej przejrzysty i funkcjonalny. Dodatkowo lekarz ma wygodny podgląd do listy wcześniej wystawionych zwolnień, jakie pacjent otrzymał w ostatnich miesiącach – tłumaczy Małgorzata Korba, regionalny rzecznik ZUS na Lubelszczyźnie.\', \'Testy przyniosły pozytywne opinie. Lekarze z regionu znajdą nowy kreator na swojej Platformie Usług Elektronicznych.\', \'Na PUE ZUS pojawiły się również inne nowe funkcje dla lekarz

In [17]:
pd.set_option('max_colwidth', 150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...
2020-02,"['Jej matka spotykała się z 22-latkiem, a on… wykorzystywał seksualnie jej córkę! Horror na Lubelszczyźnie! Śledczy podejrzewają, że cudzoziemiec ..."
2020-03,['Kolejny dramatyczny wzrost liczby chorych – w Polsce i na Lubelszczyźnie. Epidemia koronawirusa przybiera na sile. Dziesięć nowych przypadków za...
2020-04,"['Chwile grozy przeżyła 43-letnia kobieta, która wybrała się na zakupy do Włodawy (woj. lubelskie). Przeraziła się, bo ktoś ukradł jej opla zapark..."
2020-05,"['Miał zatroszczyć się o wszystko: od transportu do wspaniałego hotelu, przez skipassy, posiłki i piękne widoki na stokach. Marek P. (47 l.) z Lub..."
2020-06,"['Tomasz Pakuła z Zamościa (woj. lubelskie) zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea, coraz bardziej marzył... o własnym..."
2020-07,"['Przekonywał, że z zamiłowania jest ogrodnikiem, a ostra jak brzytwa maczeta służy mu do... przycinania żywopłotu. W rzeczywistości Piotr M. (42 ..."
2020-08,['Inspekcja Weterynaryjna wykryła Salmonellę Enteritidis w jednej z pięciu przebadanych próbek wycinków szyi kurcząt oraz w jednej z pięciu próbek...


In [18]:
data_df.transcript.loc['2020-02']

'[\'Jej matka spotykała się z 22-latkiem, a on… wykorzystywał seksualnie jej córkę! Horror na Lubelszczyźnie! Śledczy podejrzewają, że cudzoziemiec molestował 11-latkę. Piekło dziecka mogło trwać od dawna…\', \'Horror na Lubelszczyźnie, w powiecie chełmskim! Czy 22-letni cudzoziemiec molestował 11-latkę?! Mężczyzna mógł wykorzystać seksualnie córkę swojej partnerki. O sprawie informuje Super Tydzień Chełmski. Dziewczynka mogła być molestowana przez młodego mężczyznę przez ostatni rok.\', \'Polecany artykuł:\', \'Prokuratura – ze względu dobro 11-latki i jej rodziny – nie ujawnia szczegółów śledztwa. Sprawa jest bardzo delikatna.\', \'Jak informuje lokalny tygodnik, dziewczyna zwierzyła się swoim koleżankom. Powiedziała, że przez ostatni rok została kilkukrotnie wykorzystana seksualnie przez partnera swojej matki. Koleżanki powiadomiły szkołę, a ta policję.\', \'Mężczyzna został zatrzymany i przesłuchany w prokuraturze. Nie przyznaje się do winy. Ze względu na dobro śledztwa i dziewczyn

In [19]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    #text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r"\d+", "",  text)
    #text = re.sub('*d', '', text)
    #text = re.sub('"*s"', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [20]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
2020-01,prawie trzy tysiące ezwolnień wystawiają dziennie lekarze z lubelszczyzny nowy kreator do ich wypełniania od zakładu ubezpieczeń społecznych może ...
2020-02,jej matka spotykała się z latkiem a on… wykorzystywał seksualnie jej córkę horror na lubelszczyźnie śledczy podejrzewają że cudzoziemiec molestowa...
2020-03,kolejny dramatyczny wzrost liczby chorych – w polsce i na lubelszczyźnie epidemia koronawirusa przybiera na sile dziesięć nowych przypadków zakaże...
2020-04,chwile grozy przeżyła letnia kobieta która wybrała się na zakupy do włodawy woj lubelskie przeraziła się bo ktoś ukradł jej opla zaparkowanego na ...
2020-05,miał zatroszczyć się o wszystko od transportu do wspaniałego hotelu przez skipassy posiłki i piękne widoki na stokach marek p l z lublina za orga...
2020-06,tomasz pakuła z zamościa woj lubelskie zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea coraz bardziej marzył o własnym własne m...
2020-07,przekonywał że z zamiłowania jest ogrodnikiem a ostra jak brzytwa maczeta służy mu do przycinania żywopłotu w rzeczywistości piotr m l z kamionki...
2020-08,inspekcja weterynaryjna wykryła salmonellę enteritidis w jednej z pięciu przebadanych próbek wycinków szyi kurcząt oraz w jednej z pięciu próbek z...


In [21]:
data_clean.transcript.loc['2020-02']

'jej matka spotykała się z latkiem a on… wykorzystywał seksualnie jej córkę horror na lubelszczyźnie śledczy podejrzewają że cudzoziemiec molestował latkę piekło dziecka mogło trwać od dawna… horror na lubelszczyźnie w powiecie chełmskim czy letni cudzoziemiec molestował latkę mężczyzna mógł wykorzystać seksualnie córkę swojej partnerki o sprawie informuje super tydzień chełmski dziewczynka mogła być molestowana przez młodego mężczyznę przez ostatni rok polecany artykuł prokuratura – ze względu dobro latki i jej rodziny – nie ujawnia szczegółów śledztwa sprawa jest bardzo delikatna jak informuje lokalny tygodnik dziewczyna zwierzyła się swoim koleżankom powiedziała że przez ostatni rok została kilkukrotnie wykorzystana seksualnie przez partnera swojej matki koleżanki powiadomiły szkołę a ta policję mężczyzna został zatrzymany i przesłuchany w prokuraturze nie przyznaje się do winy ze względu na dobro śledztwa i dziewczynki – został aresztowany najbliższe trzy miesiące spędzi za kratami

In [22]:
next(iter(data.keys()))

'2020-01'

In [23]:
names = list(data.keys())

In [24]:
data_df['publication_month'] = names

In [25]:
data_df

,transcript,publication_month
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...,2020-01
2020-02,"['Jej matka spotykała się z 22-latkiem, a on… wykorzystywał seksualnie jej córkę! Horror na Lubelszczyźnie! Śledczy podejrzewają, że cudzoziemiec ...",2020-02
2020-03,['Kolejny dramatyczny wzrost liczby chorych – w Polsce i na Lubelszczyźnie. Epidemia koronawirusa przybiera na sile. Dziesięć nowych przypadków za...,2020-03
2020-04,"['Chwile grozy przeżyła 43-letnia kobieta, która wybrała się na zakupy do Włodawy (woj. lubelskie). Przeraziła się, bo ktoś ukradł jej opla zapark...",2020-04
2020-05,"['Miał zatroszczyć się o wszystko: od transportu do wspaniałego hotelu, przez skipassy, posiłki i piękne widoki na stokach. Marek P. (47 l.) z Lub...",2020-05
2020-06,"['Tomasz Pakuła z Zamościa (woj. lubelskie) zobaczył już prawie pół świata i odwiedzając kolejne ciekawe muzea, coraz bardziej marzył... o własnym...",2020-06
2020-07,"['Przekonywał, że z zamiłowania jest ogrodnikiem, a ostra jak brzytwa maczeta służy mu do... przycinania żywopłotu. W rzeczywistości Piotr M. (42 ...",2020-07
2020-08,['Inspekcja Weterynaryjna wykryła Salmonellę Enteritidis w jednej z pięciu przebadanych próbek wycinków szyi kurcząt oraz w jednej z pięciu próbek...,2020-08


In [26]:
data_df.to_pickle("datasets/2_corpus.pkl")

In [27]:
# from stop_words import get_stop_words

In [28]:
# stop_words = get_stop_words('polish')
# stop_words

In [29]:
stop_words

frozenset({'',
           'a',
           'aby',
           'ach',
           'acz',
           'aczkolwiek',
           'aj',
           'albo',
           'ale',
           'alez',
           'ależ',
           'and',
           'ani',
           'artykuł',
           'az',
           'aż',
           'bardziej',
           'bardzo',
           'beda',
           'bede',
           'bedzie',
           'bez',
           'bo',
           'bowiem',
           'browser',
           'by',
           'byc',
           'byl',
           'byla',
           'byli',
           'bylo',
           'byly',
           'bynajmniej',
           'być',
           'był',
           'była',
           'było',
           'były',
           'będzie',
           'będą',
           'będę',
           'cala',
           'cali',
           'caly',
           'cała',
           'cały',
           'ci',
           'cie',
           'ciebie',
           'cię',
           'co',
           'cokolwiek',
         

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,aarts,ab,abb,abigail,abisyńskaco,abisyńskie,abiturientów,abonament,abonamentowe,abonamentu,...,żądał,żądań,żądnego,żłobka,żłobkach,żłobkami,żłobki,żłobkowe,żłobków,żłopać
2020-01,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2020-02,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2020-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2,0,0,0
2020-04,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,4,0,6,0
2020-05,0,1,0,0,0,0,0,0,0,0,...,1,1,0,2,1,1,4,0,6,0
2020-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2020-07,1,0,0,0,0,0,0,0,1,2,...,2,0,0,0,0,0,1,0,1,0
2020-08,0,0,0,0,0,0,1,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [31]:
data_dtm.to_pickle("datasets/3_dtm.pkl")

In [32]:
data_clean.to_pickle('datasets/4_data_clean.pkl')
pickle.dump(cv, open("datasets/5_cv.pkl", "wb"))

In [33]:
data_dtm.sum()

aarts           1
ab              1
abb             1
abigail         1
abisyńskaco     1
               ..
żłobkami        2
żłobki         11
żłobkowe        1
żłobków        15
żłopać          1
Length: 45909, dtype: int64